staff

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("../input/train.csv")
df_x = df.drop(labels=["price_doc"], axis=1)
test = pd.read_csv("../input/test.csv")
combined = pd.concat([df_x,test], ignore_index=True, axis=0)
obj_col = combined.select_dtypes(include=[object]).columns


In [ ]:
#counts = df_x.groupby(by="sub_area").count()
#for i, name in enumerate(counts.index):
    #sub_area_row = counts.iloc[i, :]
    #zeros = sub_area_row[sub_area_row==0]
    #if not zeros.empty:
        #message="{area} doesn't have: {cols}".format(area=name, cols=zeros.index.values)
        #print(message)
    

In [ ]:

from sklearn.preprocessing import LabelEncoder
for name in obj_col:
    if name != "timestamp" and name != "product_type":
        print(name)
        encoder = LabelEncoder()
        combined[name] = encoder.fit_transform(combined[name].fillna(value=99).values)
        
        
combined["product_type"] = combined.product_type.map({"Investment":1, 
                                                      "OwnerOccupier":0, np.nan:99}).values

In [ ]:
combined.product_type.value_counts()

In [ ]:
#combined = filled_id.merge(combined["id"], right_index=True)
filled= combined.groupby(by="sub_area").fillna(99)
combined = filled.merge(combined[["id", "sub_area"]], on="id")

In [ ]:
combined["age"] = pd.to_datetime(combined["timestamp"]).dt.year - combined.build_year
combined["ratio"] = (combined["full_sq"]/combined["life_sq"]).fillna(value=99)
combined = combined.replace([-np.inf, np.inf], 99)
x_train = combined.drop(labels=["timestamp", "id"], axis=1)[0:30471].values
x_test = combined.drop(labels=["timestamp", "id"], axis=1)[30471:]
y_train = df.price_doc.values

In [ ]:
import matplotlib.pyplot as plt
_, _, hist = plt.hist([y_train, combined.full_sq[:30471].values], bins=50, label=["first", "second"])
plt.legend(loc="upper left")


In [ ]:
"""from sklearn.linear_model import RandomizedLasso
clf = RandomizedLasso(n_resampling=100)
clf.fit(X=x_train, y=y_train)
x_train = clf.transform(x_train)
x_test = clf.transform(x_test)"""

In [ ]:

"""from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(max_depth=10, min_samples_split=10, min_samples_leaf=7)
GBR.fit(X=x_train, y=y_train)
GBR.score(X=x_train, y=y_train)"""

In [ ]:
"""predictions = GBR.predict(X=x_test)
predictions = pd.DataFrame({"id":test.id, "price_doc":predictions})
predictions.to_csv("from_account_notebook.csv", index=False)"""

In [ ]:
"""
filled nans by 99
added ratio and age variables
used stability selection (randomisedlasso(n_resampling=100))to select features
selected features fed into xgb model with args:
    GradientBoostingRegressor(max_depth=10, min_samples_split=10, min_samples_leaf=7)
"""